Let's get the stats that maybe everyone can agree on as a start


In [3]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from rubin_sim.satellite_constellations import (Constellation, starlink_tles_v2,
                                                oneweb_tles,
                                                ModelObservatory,
                                                SatelliteAvoidBasisFunction)
import healpy as hp
from rubin_sim.data import get_baseline
import pandas as pd
import sqlite3
import os

In [ ]:
save_file = 'streaks_all.npz'

In [5]:
# a starlink gen 2 + oneweb
tles = starlink_tles_v2() + oneweb_tles()
const = Constellation(tles)
print('number of satellites= ', len(tles))

number of satellites=  36360


In [3]:
baseline = get_baseline()
con = sqlite3.connect(baseline)
# year 1, no twilight NEO observations
visits = pd.read_sql('select fieldRA, fieldDec, fivesigmadepth, observationstartmjd,rotSkyPos from observations where night < 365 and visitexposuretime > 20', con)
con.close()

In [4]:
visits

,fieldRA,fieldDec,fiveSigmaDepth,observationStartMJD,rotSkyPos
0,356.288005,-51.164810,23.128326,60218.004914,129.375130
1,0.158759,-53.139864,23.075882,60218.005362,130.108353
2,4.323837,-51.858683,23.147026,60218.005810,134.437221
3,4.386324,-54.962388,23.149994,60218.006256,131.810738
4,4.380452,-58.068073,23.201271,60218.006702,129.126645
...,...,...,...,...,...
44392,41.038265,-3.628448,23.930048,60582.374718,250.109668
44393,45.759568,-0.586964,23.933535,60582.375179,243.746554
44394,55.829011,2.654325,23.969454,60582.375701,230.577839
44395,60.036482,-3.725642,24.076105,60582.376165,229.067558


In [5]:
keys = ['fieldRA', 'fieldDec', 'fiveSigmaDepth', 'observationStartMJD', 'rotSkyPos']
types = [float] * len(keys)

numpy_visits = np.zeros(np.size(visits["fieldRA"].values), dtype=list(zip(keys,types)))

for key in keys:
    numpy_visits[key] = visits[key].values



In [6]:
if os.path.isfile(save_file):
    data = np.load(save_file)
    streak_lengths = data['streak_lengths'].copy()
    n_streaks = data['n_streaks'].copy()
    data.close
else:
    streak_lengths, n_streaks = const.check_pointings(visits["fieldRA"].values,
                                                         visits["fieldDec"].values,
                                                         visits["observationStartMJD"].values, visit_time=32.)

In [7]:
n_streaks

array([1, 1, 0, ..., 0, 0, 1])

In [17]:
# save so we don't have to grind that out again
if not os.path.isfile('streaks.npz'):
    np.savez('streaks.npz', streak_lengths=streak_lengths, n_streaks=n_streaks)

In [19]:
exptimes = numpy_visits['fieldRA']*0+30
exptime_before = np.sum(exptimes)
exptimes[np.where(n_streaks > 0)] -= 15
exptime_after = np.sum(exptimes)

print('Fractional loss of total exposure=', (exptime_before-exptime_after)/exptime_before)


Fractional loss of total exposure= 0.02656711039034169


In [20]:
# what fraction of pixels are lost to streaks with a 1 arcmin mask
# 
pixscale = 0.2 # arcsec/pix
tot_pix = 3.2e9 # total pixels in focal plane
mask_width = 60 # arcsec

print('fraction of pix lost', np.sum(streak_lengths/pixscale*3600 * mask_width /pixscale)/(tot_pix*np.size(streak_lengths)))

fraction of pix lost 0.0002808929369348757


In [21]:
# fraction of images with a streak
print('fraction of streaked images= ', np.where(n_streaks > 0)[0].size/np.size(n_streaks))

0.05313422078068338

In [22]:
print('mean streak length', np.mean(streak_lengths)*60, ' (arcmin)') # to arcmin

9.987304424351137